# Create Common Functions

In [ ]:
%run "./Common Functions"

## Set Client ID and Secret for Service Principal

In [ ]:
client_id = "" #dbutils.secrets.get(scope = "", key = "")
client_secret = "" #dbutils.secrets.get(scope = "", key = "")
tenant_id = "" #dbutils.secrets.get(scope = "", key = "")
authority_url = "https://login.microsoftonline.com/" + tenant_id + "/oauth2/token"
scope = "https://analysis.windows.net/powerbi/api"

## Set folder paths

In [ ]:
rawFolderName = "PowerBIActivityEvent"
curatedFolderName = "t_pbi_activity_event"
fileName = f"{rawFolderName}.json"
basePath = '/PowerBI'
rawpath = f'{basePath}/raw/{rawFolderName}'
curatedPath = f'{basePath}/curated/{curatedFolderName}'

## Set schema of json file

In [ ]:
json_schema ='''
Id string, 
CreationTime timestamp,
Activity string,
Operation string, 
UserType int, 
UserKey string,  
UserId string, 
ClientIP string, 
UserAgent string, 
IsSuccess boolean, 
ItemName string, 
CapacityName string,
WorkSpaceName string, 
DatasetName string, 
ReportName string, 
DistributionMethod string, 
ConsumptionMethod string,
FolderDisplayName string,
FolderAccessRequests string,
ModelsSnapshots string,
ReportType string, 
RecordType int,
Workload string,
ExportEventActivityTypeParameter string,
ExportEventStartDateTimeParameter string,
ExportEventEndDateTimeParameter string,
OrganizationId string, 
CapacityId string,
WorkspaceId string, 
FolderObjectId string,
ObjectId string, 
DatasetId string, 
ReportId string,
RequestId string, 
ActivityId string
'''

# Extract Access Token and Expiry time

In [ ]:
access_token, expires_at = GetAccessToken(client_id, client_secret, authority_url, scope)

# If "Curated" delta table exists then do Incremental Load else Full Load

In [ ]:
from delta.tables import DeltaTable
today = date.today()
if delta_table_exists(curatedPath): #Incremental Load
    lastDate = spark.read.format("delta").load(curatedPath).selectExpr('cast(max(CreationTime) as date)').first()[0]
    dateList = generate_date_list(lastDate, today)
    incrementPath = parallel_request(dateList, basePath, rawpath, fileName, access_token)
    if incrementPath:
        curatedTable = DeltaTable.forPath(spark, curatedPath)
        updatesDF = dfRead("json", incrementPath, json_schema)
        merge_df = (curatedTable
                    .alias("log")
                    .merge(updatesDF.alias("incr"),"log.CreationTime = incr.CreationTime AND log.Id = incr.Id") 
                    .whenNotMatchedInsertAll()
                    .execute())
else: # Full Load
    startDate = today - timedelta(days=30)
    dateList = generate_date_list(startDate, today)
    parallel_request(dateList, basePath, rawpath, fileName, access_token)
    df = (dfRead("json", rawpath, json_schema)
          .write
          .format("delta")
          .mode("overwrite")
          .option("overwriteSchema","true")
          .save(curatedPath))

# Register the table in hive metastore

In [ ]:
spark.sql(f"CREATE TABLE IF NOT EXISTS {curatedFolderName} USING DELTA LOCATION '{curatedPath}'")

Out[14]: DataFrame[]

# Create view for Reporting

In [ ]:
%sql 
CREATE OR REPLACE VIEW v_pbi_activity_event AS
select
  cast(CreationTime as date) as `Date`,
  Activity,
  UserId as `User ID`,
  CASE
    WHEN IsSuccess = true THEN 'Y'
    ELSE 'N'
  END as `Is Success`,
  WorkSpaceName as `Workspace Name`,
  DatasetName as `Dataset Name`,
  ItemName as `Item Name`,
  ReportName as `Report Name`,
  ReportType as `Report Type`,
  CapacityName as `Capacity Name`,
  DistributionMethod as `Distribution Method`,
  ConsumptionMethod as `Consumption Method`
from
  t_pbi_activity_event
where
  UserId like '%@%'

# Test Scripts

In [ ]:
rawCount = dfRead("json", rawpath, json_schema).count()
curatedCount = dfRead("delta", curatedPath).count()
if rawCount == curatedCount:
    print(f"Row Count: {curatedCount}")
else:    
    dbutils.notebook.exit("Raw and Curated counts not matching!")

In [ ]:
%sql
select max(CreationTime), count(*) from t_pbi_activity_event

In [ ]:
%sql
select count(*) from v_pbi_activity_event 